# Grocery, Market Forecasting Demo

#### Granularity: Grocery, Market
#### Frequency: Weekly
#### Evaluation: Weekly
####  Horizon: 12 Weeks

## Imports

In [ ]:
import pyodbc
from getpass import getpass

import pandas as pd
import numpy as np
import plotly

from greykite.common import constants as cst
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.framework.templates.autogen.forecast_config import (ComputationParam,
                                                                        EvaluationMetricParam,
                                                                        EvaluationPeriodParam,
                                                                        MetadataParam, ForecastConfig,
                                                                        ModelComponentsParam)
from greykite.sklearn.cross_validation import RollingTimeSeriesSplit
from greykite.framework.benchmark.benchmark_class import BenchmarkForecastConfig
from greykite.common.evaluation import EvaluationMetricEnum

import statsmodels.api as sm

from plotly.offline import init_notebook_mode, iplot

## Query

In [2]:
username = 'aspuri'
password = getpass()

host = 'tdprod1cop1.ngco.com'
driver = 'Teradata Database ODBC Driver 16.20'

link = 'DRIVER={driver};DBCNAME={host};UID={uid};PWD={pwd}'.format(
                      driver = driver, host = host,  
                      uid = username, pwd = password)

········


In [3]:
#produce sales for open sites
query = f"""
SELECT
CAL_WK_END_DT week_end_dt
, SUM(x0.PRRTD_PSTD_SL_AMT) sales
, (x1.CORP_YR_NUM + (x1.CORP_PD_NUM/13.0)) year_period
, (x1.CORP_YR_NUM + (x1.CORP_QTR_NUM/4.0)) year_quarter
FROM RLDMPROD_V.RTL_QLFY_PRRTD_PSTD_SL_DLY x0
LEFT JOIN RLDMPROD_V.CAL_DT x1
ON x0.TRANS_DT = x1.CAL_DT
LEFT JOIN CNSLD_ARTCL x2
ON TO_NUMBER(x0.SCAN_CD) = x2.SCAN_CD
LEFT JOIN CNSLD_MCH_HIER x3
ON x2.LCL_MCH_0_CD = x3.MCH_0_CD
LEFT JOIN RLDMPROD_V.SITE_HIER x4
ON x0.SITE_NUM = x4.SITE_NUM
--only keep currently open sites
WHERE x0.SITE_NUM in (
    SELECT
    SITE_NUM
    FROM
    RLDMPROD_V.SITE_HIER S
    WHERE (S.STR_CLS_DT > DATE
    OR S.STR_CLS_DT IS NULL)
    AND S.STR_OPN_DT < DATE
    AND
      (
          (S.BAN_SHRT_DESC_E in ('Fortinos','Your Ind Grocer', 'Valu-Mart', 'Atl Your Ind Grocer', 'nofrills', 'Provigo') AND PROFL_SITE_NUM in ('L160'))
          OR (S.BAN_SHRT_DESC_E in ('Loblaw', 'Zehrs', 'Atlantic Superstore', 'Superstore', 'Wholesale Club', 'Dominion', 'Maxi', 'Retail RCWC', 'Extra Foods') AND PROFL_SITE_NUM in ('L130'))
      ) 
    --this list discludes Joe Fresh and Liquorstore
    --Your Ind Grocer | Ontario has 5 extra sites...4987,4988,4989,4990,4991
    --Provigo | Quebec has 4 extra sites...4549, 4913, 4914, 4934
    --Extra Foods | West  has 3 extra sites...4982, 4983, 4984
    AND x0.SITE_NUM NOT IN ('4987', '4988', '4989', '4990', '4991', '4549', '4913', '4914', '4934', '4982', '4983', '4984')
)
AND x3.MCH_3_ENG_DESC IN ('Bakery', 'Deli', 'Ancillary', 'Centre of Store', 'HMR', 'Cosmetics')
AND x4.CUST_GRP_ENG_DESC = 'Discount'
AND x1.CAL_YR_NUM > 2017
AND (
    (x1.CAL_YR_NUM <> YEAR(DATE)) OR
    (x1.CORP_WK_NUM <> (SELECT CORP_WK_NUM FROM RLDMPROD_V.CAL_DT WHERE CAL_DT = DATE))
    )
GROUP BY CAL_WK_END_DT, year_period, year_quarter
ORDER BY CAL_WK_END_DT
"""

with pyodbc.connect(link,autocommit=True) as connect:
    df = pd.read_sql(query,connect)

In [5]:
ts = UnivariateTimeSeries()
ts.load_data(
    df=df,
    time_col="week_end_dt",
    value_col="sales",
    freq="W-SAT") 


init_notebook_mode(connected=True) 

fig = ts.plot()
iplot(fig)

## Preprocess

In [6]:
# when adding lags, the lag coefficient will be small (~1) while other coefficients will be in the millions
# so scaling prevents numerical issues
scale = df['sales'].max()
df['sales'] = df['sales']/scale

### Non-Model Configuration

In [7]:
# first we define the non model components

# forecast horizon
# the cv will use this horizon to compute performance metrics
forecast_horizon = 8

# the coverage of the confidence intervals
# none means no confidence intervals (saves on computation)
coverage = None

# defining anomalies
# anomalies will not be imputed (they will be completely ommited)
anomaly_df = pd.DataFrame({
 # start and end date are inclusive
 cst.START_DATE_COL: ["2020-03-14"],  # inclusive
 cst.END_DATE_COL: ["2020-05-23"],  # inclusive
 cst.ADJUSTMENT_DELTA_COL: [np.nan],  # mask as NA
 cst.METRIC_COL: ["sales"]
})

# the time series metadata
metadata = MetadataParam(
        time_col = 'week_end_dt', 
        value_col = 'sales',
        freq = 'W-SAT', # the time series is weekly with weeks starting on Sat
        anomaly_info = {
         "value_col": 'sales',
         "anomaly_df": anomaly_df,
         "adjustment_delta_col": cst.ADJUSTMENT_DELTA_COL,
        }
)


# holiday variables, with additional lead variables
# for daily data greykite will automatically include holidays depending on the country specified
# this hasnt been implemented for weekly data so we do this manually
events = dict(
    holiday_lookup_countries = None,
        daily_event_df_dict = {
        "easter": pd.DataFrame({
            "date": ["2018-03-31", "2019-04-20", "2020-04-11", "2021-04-03", "2020-04-16"],
            "event_name": "is_event"
        }),
        "easterlead": pd.DataFrame({
            "date":["2018-04-07", "2019-04-27", "2020-04-18", "2021-04-10", "2020-04-23"],
            "event_name": "is_event2"
        }),
        "thanksgiving": pd.DataFrame({
            "date": ["2018-10-06", "2019-10-12", "2020-10-10", "2021-10-09", "2020-10-08"],
            "event_name": "is_event3"
        }),
        "thanksgivinglead": pd.DataFrame({
            "date":["2018-10-13", "2019-10-19", "2020-10-17", "2021-10-06", "2020-10-15"],
            "event_name": "is_event4"
        }),
    }
)  


# the computational metrics (e.g. how many cores to use)
computation = ComputationParam(
    n_jobs = 1,
    verbose = 1
)


# defines the cross-validation configuration
# i'm using a different cross-validation function they provide, but still have to include this
evaluation_period = EvaluationPeriodParam(
    test_horizon = 0,
    cv_max_splits = 1,
    periods_between_train_test = 0
)

 # defines the cross validation parameters
tscv = RollingTimeSeriesSplit(
    forecast_horizon = forecast_horizon,
    min_train_periods = 128, # 1 week after anomaly ends, so we have 1 lag
    expanding_window = True,
    use_most_recent_splits = True,
    periods_between_splits = 3,
    max_splits = 18
)

 # Print the train, test split for BM folds
# for split_num, (train, test) in enumerate(tscv.split(X=df)):
#     print(split_num, train, test)   

metric_dict = {
     "MAPE": EvaluationMetricEnum.MeanAbsolutePercentError,
     "RMSE": EvaluationMetricEnum.RootMeanSquaredError
 }


### Model Configuration

In [8]:
#last we define the model components

# normal yearly seasonlity, no trend changepoints
SK1_param = ModelComponentsParam(
    events = events,
    custom = {'min_admissible_value': 0,
             'fit_algorithm_dict': {'fit_algorithm': 'statsmodels_ols'}
             },
    growth = {'growth_term': 'linear'},
    seasonality = {'yearly_seasonality': 15,
                   'quarterly_seasonality': False,
                   'monthly_seasonality': False},
    changepoints = {'changepoints_dict': None},
    autoregression = {'autoreg_dict': None}
)

# normal yearly seasonlity, light trend changepoints
SK2_param =  ModelComponentsParam(
    events = events,
    custom = {'min_admissible_value': 0,
             'fit_algorithm_dict': {'fit_algorithm': 'statsmodels_ols'}
             },
    growth = {'growth_term': 'linear'},
    seasonality = {'yearly_seasonality': 15,
                   'quarterly_seasonality': False,
                   'monthly_seasonality': False},
    changepoints = {'changepoints_dict':
                    {'method': 'auto',
                     'resample_freq': '7D',
                     'regularization_strength': 0.6,
                     'potential_changepoint_distance': '14D',
                     'no_changepoint_distance_from_end': '90D', 
                     'yearly_seasonality_order': 15, 
                     'yearly_seasonality_change_freq': None}
                   },
    autoregression = {'autoreg_dict': None}
)

# heavy yearly seasonlity, normal trend changepoints
SK3_param =  ModelComponentsParam(
    events = events,
    custom = {'min_admissible_value': 0,
             'fit_algorithm_dict': {'fit_algorithm': 'ridge'}
             },
    growth = {'growth_term': 'linear'},
    seasonality = {'yearly_seasonality': 25,
                   'quarterly_seasonality': False,
                   'monthly_seasonality': False},
    changepoints = {'changepoints_dict':
                    {'method': 'auto',
                     'resample_freq': '7D',
                     'regularization_strength': 0.5,
                     'potential_changepoint_distance': '14D',
                     'no_changepoint_distance_from_end': '90D', 
                     'yearly_seasonality_order': 15, 
                     'yearly_seasonality_change_freq': '365D'}
                   },
    autoregression = {'autoreg_dict': None}
)

# heavy yearly seasonality, light trend changepoints
SK4_param =  ModelComponentsParam(
    events = events,
    custom = {'min_admissible_value': 0,
             'fit_algorithm_dict': {'fit_algorithm': 'ridge'}
             },
    growth = {'growth_term': 'linear'},
    seasonality = {'yearly_seasonality': 25,
                   'quarterly_seasonality': False,
                   'monthly_seasonality': False},
    changepoints = {'changepoints_dict':
                    {'method': 'auto',
                     'resample_freq': '7D',
                     'regularization_strength': 0.6,
                     'potential_changepoint_distance': '14D',
                     'no_changepoint_distance_from_end': '90D', 
                     'yearly_seasonality_order': 15, 
                     'yearly_seasonality_change_freq': None}
                   },
    autoregression = {'autoreg_dict': None},
)


# heavy yearly seasonality, light trend changepoints, 4 week averaged ar
SK5_param =  ModelComponentsParam(
    events = events,
    custom = {'min_admissible_value': 0,
             'fit_algorithm_dict': {'fit_algorithm': 'ridge'}
             },
    growth = {'growth_term': 'linear'},
    seasonality = {'yearly_seasonality': 25,
                   'quarterly_seasonality': False,
                   'monthly_seasonality': False},
    changepoints = {'changepoints_dict':
                    {'method': 'auto',
                     'resample_freq': '7D',
                     'regularization_strength': 0.6,
                     'potential_changepoint_distance': '14D',
                     'no_changepoint_distance_from_end': '90D', 
                     'yearly_seasonality_order': 15, 
                     'yearly_seasonality_change_freq': None}
                   },
    autoregression = {'autoreg_dict': {'agg_lag_dict': {'orders_list': [[1,2,3,4]]}}}
)

In [13]:
#then we create the final configs
SK1_config = ForecastConfig(
            forecast_horizon = forecast_horizon,  
            coverage = coverage,        
            metadata_param = metadata,
            computation_param = computation,
            evaluation_period_param = evaluation_period,
            model_components_param = SK1_param)

SK2_config = ForecastConfig(
            forecast_horizon = forecast_horizon,  
            coverage = coverage,        
            metadata_param = metadata,
            computation_param = computation,
            evaluation_period_param = evaluation_period,
            model_components_param = SK2_param)

SK3_config = ForecastConfig(
            forecast_horizon = forecast_horizon,  
            coverage = coverage,        
            metadata_param = metadata,
            computation_param = computation,
            evaluation_period_param = evaluation_period,
            model_components_param = SK3_param)

SK4_config = ForecastConfig(
            forecast_horizon = forecast_horizon,  
            coverage = coverage,        
            metadata_param = metadata,
            computation_param = computation,
            evaluation_period_param = evaluation_period,
            model_components_param = SK4_param)

SK5_config = ForecastConfig(
            forecast_horizon = forecast_horizon,  
            coverage = coverage,        
            metadata_param = metadata,
            computation_param = computation,
            evaluation_period_param = evaluation_period,
            model_components_param = SK5_param)

In [14]:
configs = {'SK1': SK1_config, 'SK2': SK2_config, 'SK3': SK3_config, 'SK4': SK4_config, 'SK5': SK5_config}

In [15]:
# running the CV
bm = BenchmarkForecastConfig(df = df, configs = configs, tscv = tscv)
bm.run()
bm.extract_forecasts()

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\sklearn\cross_validation.py:196: UserWarning:

There is a high number of CV splits (18). If training is slow, increase `periods_between_splits` or `min_train_periods`, or decrease `max_splits`



C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\framework\pipeline\pipeline.py:181: UserWarning:

No data selected for test (test_horizon=0). It is important to check out of sample performance

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\sklearn\cross_validation.py:194: UserWarning:

There is only one CV split

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps



Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are 

Fitting 1 folds for each of 1 candidates, totalling 1 fits


...

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are 

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are 

Fitting 1 folds for each of 1 candidates, totalling 1 fits


C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:2054: UserWarning:

The granularity of data is larger than daily. Ensure the daily events data match the timestamps

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are 

In [16]:
evaluation_metrics = bm.get_evaluation_metrics(metric_dict = metric_dict)
errors = evaluation_metrics.drop(columns=["split_num"]).groupby("config_name", as_index = False).mean()

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\ev

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\evaluation.py:90: UserWarning:

11 value(s) in y_true were NA or infinite and are omitted in error calc.

C:\Users\aspuri\AppData\Roaming\Python\Python37\site-packages\greykite\common\ev

In [17]:
# the cv metrics
errors

,config_name,train_MAPE,test_MAPE,train_RMSE,test_RMSE
0,SK1,2.293744,2.642816,0.019779,0.022142
1,SK2,2.130029,2.817018,0.018903,0.024326
2,SK3,1.838350,2.685650,0.016052,0.022275
3,SK4,1.860584,2.835993,0.016089,0.023573
4,SK5,1.543745,2.720755,0.013846,0.023161


In [18]:
bestModelRMSE = errors.test_RMSE.min()
bestModelName = errors[errors['test_RMSE'] == bestModelRMSE]['config_name'].values[0]

bestModelMAPE = errors[errors['config_name'] == bestModelName]['test_RMSE'].values[0]

rolling_forecasts = bm.result[bestModelName]["rolling_forecast_df"]

In [19]:
model = bm.result[bestModelName]['rolling_evaluation']['split_12']["pipeline_result"].model
summary = model[-1].summary()

In [20]:
# the model summary of the best model

summary

================================ Model Summary =================================

Number of observations: 182,   Number of features: 36
Method: Ordinary least squares
Number of nonzero features: 36

Residuals:
         Min           1Q       Median           3Q          Max
    -0.06302     -0.01057    0.0009666      0.01101      0.08484

           Pred_col   Estimate Std. Err  t value Pr(>|t|) sig. code                  95%CI
          Intercept     0.5615 0.003599    156.0   <2e-16       ***       (0.5543, 0.5686)
      events_easter    0.04451  0.01383    3.218    0.002        **     (0.01717, 0.07185)
  events_easterlead   -0.04273  0.01383   -3.089    0.002        **   (-0.07006, -0.01539)
events_thanksgiving    0.06586  0.02403    2.741    0.007        **      (0.01836, 0.1134)
events_t...vinglead   -0.01505  0.02403   -0.626    0.532              (-0.06255, 0.03245)
                ct1    0.04615 0.001733    26.63   <2e-16       ***     (0.04273, 0.04958)
    sin1_ct1_yearly  0